In [2]:
import importlib
from collections import namedtuple as nt

In [164]:
import pandas as pd
import importlib

import mosek.fusion as mf
from mosek import callbackcode, dinfitem, iinfitem, liinfitem

expr = mf.Expr
dom = mf.Domain
mat = mf.Matrix


np.random.seed(1)

In [187]:
n = 20
m = 10
D = np.random.randint(1, n, size=n)
A = np.random.random((m, n))
b = np.ones(m) * n
c = np.random.random(n)
d = c/2

args = A, b, c, d, m, n, D

In [188]:
D

array([ 8,  6,  3,  8,  1, 12, 10,  1,  8, 15, 16, 16, 18, 13, 18,  1, 12,
        3, 14,  6])

## Benchmark by MILP

In [218]:
def full_model(*args):
  model = mf.Model('full')
  y = model.variable("y", n, dom.greaterThan(0))
#   y.makeInteger()
  delta = model.variable("delta", n, dom.greaterThan(0))
  eps = model.variable("eps", n, dom.greaterThan(0))
  # constraints
  model.constraint(expr.sub(expr.add(y, delta), eps), dom.equalsTo(D.astype(float)))
  model.constraint(expr.mul(A, y), dom.lessThan(b))
  obj = expr.add(expr.dot(delta, c), expr.dot(eps, d))
  model.objective(mf.ObjectiveSense.Minimize, obj)
  model.setLogHandler(sys.stdout)
  model.acceptedSolutionStatus(mf.AccSolutionStatus.Anything)
  model.solve()
  model.flushSolutions()
  return y.level(), delta.level(), eps.level(), model.primalObjValue()

def sub_model(l, *args):
  A, b, c, d, m, n, D = args
  model = mf.Model('full')
  y = model.variable("y", n, dom.greaterThan(0))
#   y.makeInteger()
  # constraints
  model.constraint(expr.mul(A, y), dom.lessThan(b))
  obj = expr.sub(expr.dot(l, y), l.dot(D))
  model.objective(mf.ObjectiveSense.Minimize, obj)
  model.acceptedSolutionStatus(mf.AccSolutionStatus.Anything)
  model.solve()
  model.flushSolutions()
  return y.level(), model.primalObjValue()

In [219]:
y_sol, *_ = full_model(*args)
y_sol, *_

Problem
  Name                   : full            
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 30              
  Cones                  : 0               
  Scalar variables       : 61              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : full            
  Objective sense        : min             
  Type                   : LO (linear optimization 

(array([ 0.        ,  0.        ,  3.        ,  0.        ,  1.        ,
         0.        , 10.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  6.27858319, 16.49339575,
         0.        ,  4.62573178,  0.        ,  0.        ,  0.        ]),
 array([ 8.        ,  6.        , -0.        ,  8.        , -0.        ,
        12.        , -0.        ,  1.        ,  8.        , 15.        ,
        16.        , 16.        , 18.        ,  6.72141681,  1.50660425,
         1.        ,  7.37426822,  3.        , 14.        ,  6.        ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 50.59798312720183)

# subgrad + lagrangian heur

In [220]:
A, b, c, d, m, n, D = args

In [221]:
sol_container = nt("sol", ["primal_sol", "primal_val", "lb"])
sol_container.primal_sol = []
sol_container.primal_val = []
sol_container.lb = []
l0 = np.ones(n) * 0.67

l_bar = l_k = l0.copy()
x_bar = 0
z_bar = 0
z_star = worst = c.dot(D)
gap = 0.005
alp = alp_max = 0.1
r0 = 1
lb = -1e6
improved = 0


for k in range(0, 100):
  
  if (k + 1) % 50 == 0:
    alp_max *= 0.5
  
  # solve relaxation
  x_k, phi_k = sub_model(l_k, *args)
  x_bar = (1 - alp) * x_bar + x_k * (alp)
  
  # subgrad & heur
  surplus = (x_k - D)
  surplus_idx = surplus > 0
  z_k = (d * surplus)[surplus_idx].sum() \
    - (c * surplus)[~surplus_idx].sum()
  
  z_bar = (1 - alp) * z_bar + z_k * (alp)

  # compute multipliers from l_b, t_b
  partial_lambda = x_bar - D
  step = 1 / (partial_lambda.dot(partial_lambda)) * (worst - lb) * r0

  l_k = l_bar + step * partial_lambda

  l_k = np.minimum(np.maximum(l_k, -c), d)

  # eval
  if phi_k > lb:
    lb = phi_k
    l_bar = l_k.copy()
    improved = 0
  else:
    improved += 1
    if improved >= 30:
      r0 = r0 / 2
      improved = 0

  # update alp

  print(
    f"k: {k} @z_k: {z_k}; @z_bar: {z_bar}; @lb: {lb};\n"
    f"@stepsize: {step}; @norm: {np.abs(partial_lambda).sum()}; @alp: {alp}"
  )
  sol_container.primal_sol = x_bar
  sol_container.primal_val.append(z_bar)
  sol_container.lb.append(lb)
  
  if abs(z_bar - lb) / abs(z_bar) < gap:
    break

k: 0 @z_k: 82.03020131756017; @z_bar: 8.203020131756018; @lb: -126.63;
@stepsize: 412.7453694598917; @norm: 189.0; @alp: 0.1
k: 1 @z_k: 57.71511192210829; @z_bar: 13.154229310791246; @lb: 46.02796427743557;
@stepsize: 0.08986338728852296; @norm: 184.30254198007867; @alp: 0.1
k: 2 @z_k: 57.71511192210829; @z_bar: 17.610317571922952; @lb: 46.02796427743557;
@stepsize: 0.016064291210729265; @norm: 180.0748297621495; @alp: 0.1
k: 3 @z_k: 57.71511192210829; @z_bar: 21.620797006941487; @lb: 46.02796427743557;
@stepsize: 0.01654093755432319; @norm: 176.88100460527951; @alp: 0.1
k: 4 @z_k: 57.71511192210829; @z_bar: 25.23022849845817; @lb: 46.074451075588506;
@stepsize: 0.016941828512244635; @norm: 175.23497226478094; @alp: 0.1
k: 5 @z_k: 57.71511192210829; @z_bar: 28.478716840823186; @lb: 46.21413818446413;
@stepsize: 0.01725275039950145; @norm: 173.75354315833215; @alp: 0.1
k: 6 @z_k: 57.71511192210829; @z_bar: 31.4023563489517; @lb: 46.53072182736862;
@stepsize: 0.017458391066337984; @norm:

In [222]:
x_bar

array([ 0.        ,  0.        ,  4.86954576,  0.        ,  2.37163463,
        0.        , 10.40186645,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  2.11034854, 14.799011  ,
        0.        ,  2.2623824 ,  0.        ,  0.        ,  0.        ])

In [215]:
y_sol

array([ 0.,  0.,  3.,  0.,  1.,  0.,  6.,  0.,  0.,  0.,  0.,  0.,  1.,
        7., 15.,  0.,  8.,  0.,  0.,  0.])

In [216]:
D

array([ 8,  6,  3,  8,  1, 12, 10,  1,  8, 15, 16, 16, 18, 13, 18,  1, 12,
        3, 14,  6])

In [217]:
c

array([0.48148581, 0.19203127, 0.84711292, 0.50337761, 0.52716018,
       0.28807959, 0.51199923, 0.00196365, 0.27729504, 0.34876298,
       0.28395089, 0.08595223, 0.52178121, 0.78847913, 0.94259844,
       0.49728979, 0.59375386, 0.29085994, 0.14915951, 0.13187169])